<a href="https://colab.research.google.com/github/toruyoshida-fuller/scikit_handbook/blob/master/client-report/appmedia/game_category_rate_202101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# App media 追加データ　ゲームのサブカテゴリMAU率

## Import

In [ ]:
from google.cloud import bigquery
import pandas as pd
import numpy as np
from google.colab import auth
import gspread
import gspread_dataframe
from oauth2client.client import GoogleCredentials

project_id = "fl-gerber-bigquery"

## auth

In [ ]:
from google.colab import auth

auth.authenticate_user()

## parameter

月次データ

In [ ]:
MONTH =["2020-01-01","2020-02-01","2020-03-01","2020-04-01","2020-05-01","2020-06-01","2020-07-01","2020-08-01","2020-09-01","2020-10-01","2020-11-01","2020-12-01"]

## Query

In [ ]:
cols =["month","category1","value_sum","total_value","rate"]
df1 =pd.DataFrame(index=[], columns=cols)
for month in MONTH:
    df_game_category = pd.io.gbq.read_gbq(
        query=f"""
        WITH
  sampling_sample_user AS(
  SELECT
    DATE(datetime,'Asia/Tokyo') AS month,
    device_id,
    sex,
    age
  FROM
    `fl-gerber-bigquery.jp_gerber_dataflow.android_monthly_sampling_sample_user`
  WHERE
    DATE(_PARTITIONTIME) >= "{month}"
    AND DATE(_PARTITIONTIME) < (DATE_ADD("{month}",INTERVAL 1 month))
    ),
  sampling_user_count_table AS (
  SELECT
    month,
    COUNT(device_id) AS device_cnt
  FROM
    sampling_sample_user
  GROUP BY
    month ),
  sampling_user_count_table_sum AS (
  SELECT
    SUM(device_cnt) AS device_sum
  FROM
    sampling_user_count_table ),
  convert_store_info_v2 AS (
  SELECT
    app_id AS package_name,
    CASE
      WHEN ARRAY_LENGTH(categories)>=1 THEN categories[ OFFSET (0)]
  END
    category1,
    CASE
      WHEN ARRAY_LENGTH(categories)>=2 THEN categories[ OFFSET (1)]
  END
    category2,
    CAST( total_downloads_lower AS int64 ) AS total_downloads_lower,
    SAFE_CAST(inapp_purchase_lower AS int64 ) AS inapp_purchase_lower,
    SAFE_CAST(inapp_purchase_upper AS int64 ) AS inapp_purchase_upper,
    name,
    developer_name,
    developer_url,
    developer_email,
    developer_address,
    icon_url,
    description
  FROM
    `fl-gerber-bigquery.jp_informant_v2.googleplay_store`
  WHERE
    DATE(_PARTITIONTIME ) = (DATE_ADD("{month}",INTERVAL 1 month )) ),
  sampling_sample_coefficient AS(
  SELECT
    DATE(datetime,'Asia/Tokyo') AS month,
    sex,
    age,
    value
  FROM
    `fl-gerber-bigquery.jp_gerber_dataflow.android_monthly_sampling_coefficient`
  WHERE
    DATE(_PARTITIONTIME) >= "{month}"
    AND DATE(_PARTITIONTIME) < (DATE_ADD("{month}",INTERVAL 1 month))),
  sampling_sample_launch AS(
  SELECT
    DATE(datetime,'Asia/Tokyo') AS month,
    device_id,
    package_name
  FROM
    `fl-gerber-bigquery.jp_gerber_dataflow.android_monthly_sampling_sample_launch`
  WHERE
    DATE(_PARTITIONTIME) >= "{month}"
    AND DATE(_PARTITIONTIME) < (DATE_ADD("{month}",INTERVAL 1 month))
  GROUP BY
    month,
    device_id,
    package_name),
  sampling_sample_launch_store AS (
  SELECT
    month,
    device_id,
    category1,
    package_name
  FROM
    sampling_sample_launch
  JOIN
    convert_store_info_v2
  USING
    (package_name) ),
  sampling_sample_launch_weighted AS (
  SELECT
    month,
    device_id,
    package_name,
    category1,
    sex,
    age,
    value
  FROM
    sampling_sample_launch_store
  JOIN
    sampling_sample_user
  USING
    (month,
      device_id)
  JOIN
    sampling_sample_coefficient
  USING
    (month,
      sex,
      age)
  WHERE
    NOT package_name IN ('bimz.ux.panelOverlay',
      'bitsie.playmee.musicplayer.free',
      'co.testee.android',
      'com.blackout',
      'com.crimsonscythe.proximityunlock',
      'com.dbd.pdfcreator',
      'com.imovieCYH666',
      'com.kdanmobile.android.pdfreader.google.pad',
      'com.mkt_apps.absurvey',
      'com.moon.iphone.lockscreen',
      'com.necta.wifimousefree',
      'com.pindrop.music',
      'com.treydev.mns',
      'com.treydev.msb',
      'com.treydev.msb.pro',
      'de.android.telnet',
      'de.android.wifiscanner',
      'de.android.wifiscannes',
      'forinnovation.phoneaddiction',
      'forinnovation.phoneaddictioo',
      'jp.co.fuller.catenary.android',
      'jp.co.fuller.catenary.au.kids',
      'jp.co.fuller.cognitrip',
      'jp.co.fuller.munai_kuma_battery',
      'jp.co.fuller.munai_kuma_qr',
      'jp.co.fuller.octettruss.android',
      'jp.co.fuller.ojisancollection2.mobile',
      'jp.co.fuller.qr_ojisan_android',
      'jp.co.fuller.trimtab.x.android.couple',
      'jp.co.fuller.trimtab.x.android.gamer',
      'jp.co.fuller.trimtab.y.android.power.white',
      'jp.co.fuller.trimtab_android',
      'jp.co.fuller.wmeteor.ffbe',
      'kr.co.rinasoft.howuse',
      'wsm.wifimousefree')
  GROUP BY
    month,
    device_id,
    package_name,
    category1,
    sex,
    age,
    value ),
  sampling_sample_launch_weighted_count AS (
  SELECT
    month,
    category1,
    SUM(value) AS value_sum
  FROM
    sampling_sample_launch_weighted
  WHERE
    category1 LIKE '%GAME%'  ---ここでジャンル指定する。
  GROUP BY
    month,
    category1),
  value_sum_total AS (
  SELECT
    SUM (value_sum) AS total_value
  FROM
    sampling_sample_launch_weighted_count ),
  category_table AS (
  SELECT
    month,
    category1,
    value_sum,
    total_value,
    value_sum / total_value AS rate
  FROM
    sampling_sample_launch_weighted_count
  CROSS JOIN
    value_sum_total )
SELECT
  *
FROM
  category_table
  ORDER BY rate desc
  """,
    project_id =project_id,
    )
    df1 = df1.append(df_game_category)  

In [ ]:
df1

,month,category1,value_sum,total_value,rate
0,2020-01-01,GAME_PUZZLE,17867.746010,74819.941871,0.238810
1,2020-01-01,GAME_ROLE_PLAYING,9781.612689,74819.941871,0.130735
2,2020-01-01,GAME_ACTION,7958.224697,74819.941871,0.106365
3,2020-01-01,GAME_CASUAL,6877.889203,74819.941871,0.091926
4,2020-01-01,GAME_SIMULATION,6486.018238,74819.941871,0.086688
...,...,...,...,...,...
12,2020-12-01,GAME_CASINO,1671.469352,60882.915992,0.027454
13,2020-12-01,GAME_EDUCATIONAL,935.131683,60882.915992,0.015360
14,2020-12-01,GAME_TRIVIA,705.951054,60882.915992,0.011595
15,2020-12-01,GAME_WORD,465.683328,60882.915992,0.007649


## Output

In [ ]:
gc = gspread.authorize(GoogleCredentials.get_application_default())
ss = gc.open_by_url(
    "https://docs.google.com/spreadsheets/d/1OOrWtPMdwnGefEufVrMn2_LosvvUmDuJBcJvX3h-4wg/edit#gid=0"  # アウトプットシート
)
gspread_dataframe.set_with_dataframe(ss.get_worksheet(0), df1)